In [1]:
import os
import sys
import time
import smbus

from math import log
from time import sleep
from datetime import datetime

import matplotlib.pyplot as plt

from meerkat import ads, thermistor

In [4]:
i2c = smbus.SMBus(1)
dev = ads.ADS1115(bus=i2c)
dev.get_config()
dev.pga('6.144')

NameError: name 'smbus' is not defined

In [ ]:
dev.print_attributes()

In [ ]:
def get_time():
    return datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')

In [ ]:
def R2(R1, Vin, Vout):
    """Voltage divider, unknown resistance with known resistance and voltages"""
    return R1 * (1 / ((Vin/Vout) - 1))

In [ ]:
# delay between reads and writes sometimes need to be increased
dev.delay = 0.1

# get voltages of pins
dev.mux('1G')     # pin 0 relative to ground
Vcc = dev.voltage()
time.sleep(1)
print('Measured Source voltage: {} V'.format(Vcc))
dev.mux('2G')     # pin 1 relative to ground
Vt = dev.voltage()
print('Measured Thermistor voltage: {} V'.format(Vt))

# calculate resistance of thermistor
Rt = R2(R1=10000, Vin=Vcc, Vout=Vt)
print('Measured resistance: {} ohms'.format(Rt))

# Thermistor is a Semitec 103AT-2 values from datasheet
# http://www.semitec.co.jp/uploads/english/sites/2/2017/03/P12-13-AT-Thermistor.pdf

# these steps would be done once
s103AT = thermistor.Semitec103AT()
A = thermistor.compose_array(*s103AT.find_nearest(Rt))
abc = thermistor.gauss(A)

tC = steinhart_hart_equation(Rt, *abc)
print('Measured temperature: {:6.2f} C'.format(tC))